In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import math
import seaborn as sn
import xgboost as xgb
from sklearn import metrics
from sklearn.calibration import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('train.csv')
valid = pd.read_csv('valid.csv')
test = pd.read_csv('test.csv')

In [4]:
# Separate features and labels
X_train = train.drop(['label_1', 'label_2', 'label_3', 'label_4'], axis=1)
y_train = train[['label_1', 'label_2', 'label_3', 'label_4']]
X_val = valid.drop(['label_1', 'label_2', 'label_3', 'label_4'], axis=1)
y_val = valid[['label_1', 'label_2', 'label_3', 'label_4']]
X_test = test.drop(['ID'], axis=1)

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [6]:
cross_val_score(SVC(C=1000, class_weight='balanced'), X_train_scaled, y_train['label_4'], cv=2, scoring='accuracy').mean()

0.7982117812061711

In [8]:
pca = PCA(n_components=0.95, svd_solver = 'full')
X_train_pca = pca.fit_transform(X_train_scaled)
X_val_pca = pca.transform(X_val_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [6]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

halving_cv = HalvingGridSearchCV(SVC(), {
    'C': [1, 10,100,1000,1500,2000],
    'kernel': ['rbf'],
    'gamma': ['scale','auto', 0.1, 0.01, 0.001],
    'class_weight': ['balanced']
})
halving_cv.fit(X_train_scaled, y_train['label_4'])

print("Best Params", halving_cv.best_params_)
print("Best CV Score", halving_cv.best_score_)

Best Params {'C': 1000, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf'}
Best CV Score 0.9209400210452472


In [ ]:
from sklearn.metrics import classification_report

def get_score(model, X_train, y_train, X_val, y_val, verbose = False):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    if verbose:
        print(classification_report(y_val, y_pred))

    return accuracy_score(y_val, y_pred) * 100

In [ ]:
get_score(
    SVC(class_weight='balanced'),
    X_train_pca, y_train['label_4'],
    X_val_pca, y_val['label_4']
)

In [ ]:
# param_dist = { "learning_rate": np.linspace(0,0.2,5),
#                "max_depth": randint(3, 10)}
               
# #Instantiate RandomSearchCV object
# rscv = RandomizedSearchCV(CatBoostClassifier(loss_function='MultiClass'), param_dist, scoring='accuracy', cv =5)

# #Fit the model
# rscv.fit(X_train_pca, y_train['label_4'])

# print(rscv.best_params_)
# print(rscv.best_score_)

In [ ]:
get_score(
    CatBoostClassifier(
        loss_function='MultiClass', learning_rate=0.15, max_depth=6, task_type="GPU", devices='0:1',
        verbose=0
    ),
    X_train_pca, y_train['label_4'],
    X_val_pca, y_val['label_4']
)

In [ ]:
cross_val_score(
    CatBoostClassifier(
        loss_function='MultiClass', learning_rate=0.15, max_depth=6, task_type="GPU", devices='0:1',
        verbose=0, auto_class_weights = 'Balanced'
    ),
    X_train_pca, y_train['label_4'], cv=5
).mean() * 100

In [ ]:
cross_val_score(KNeighborsClassifier(n_neighbors=5), X_train_pca, y_train['label_4'], cv=5).mean() * 100

In [ ]:
cross_val_score(SVC(class_weight='balanced'), X_train, y_train['label_4'], cv=5, verbose=3).mean() * 100

In [ ]:
cross_val_score(SVC(C=1000, class_weight='balanced'), X_train, y_train['label_4'], cv=5, verbose=3).mean() * 100

In [ ]:
# best_model_label_4 = SVC(class_weight='balanced')